# HW3 Analysis

## Import and load data

In [13]:
import ml_pipeline as pp
import pandas as pd
import datetime as dt
import numpy as np
#from sklearn import metrics
file = './data/projects_2012_2013.csv'
df = pp.load_csv(file)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Data transformations

### Helper functions for hw3 specific data

#### Convert date columns to datetime

In [69]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'teacher_prefix', 'primary_focus_subject',
       'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area',
       'resource_type', 'poverty_level', 'grade_level',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'date_posted', 'datefullyfunded',
       'label'],
      dtype='object')

In [14]:
df.date_posted = pp.col_datetime(df, 'date_posted')

In [15]:
df.datefullyfunded = pp.col_datetime(df,'datefullyfunded')

#### Create labels: 1 if not fully funded in 60 days, 0 if fully funded in 60 days

In [16]:
df = pp.create_label(df, pred_time=60)

In [17]:
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded,label
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,2013-04-14,2013-05-02,0
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,2012-04-07,2012-04-18,0
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,2012-01-30,2012-04-15,1
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,2012-10-11,2012-12-05,0
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,2013-01-08,2013-03-25,1


## Selecting features and cleaning

In [18]:
feature_cols=['school_metro','school_charter', 'school_magnet', 'primary_focus_subject', 'primary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'total_price_including_optional_support', 'students_reached', 'eligible_double_your_impact_match', 'date_posted', 'label']
sel = df[feature_cols].copy()
sel.head()

,school_metro,school_charter,school_magnet,primary_focus_subject,primary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,label
0,urban,f,f,Mathematics,Math & Science,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,2013-04-14,0
1,urban,f,f,Civics & Government,History & Civics,Books,highest poverty,Grades 3-5,282.47,28.0,t,2012-04-07,0
2,urban,f,f,Literacy,Literacy & Language,Technology,high poverty,Grades 3-5,1012.38,56.0,f,2012-01-30,1
3,urban,f,t,Literacy,Literacy & Language,Books,high poverty,Grades PreK-2,175.33,23.0,f,2012-10-11,0
4,suburban,f,f,Literacy,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,2013-01-08,1


#### Identify feature columns with null values

In [19]:
for x in pp.na_col(df):
    if x in feature_cols:
        print(x)

school_metro
primary_focus_subject
primary_focus_area
resource_type
grade_level
students_reached


#### Impute missing categorical variables with the most frequent, which is a common way to handle missing categorical data without more information. 


In [20]:
cat_cols = ['school_metro','primary_focus_subject','primary_focus_area','resource_type','grade_level']
for x in cat_cols:
    sel = pp.na_fill_col(sel, x , pp.most_freq)

#### Impute missing numerical variable (students_reached) with the median value because there are outliers affecting the mean. 

In [21]:
sel.students_reached.quantile([0.1, 0.25, 0.5, 0.75, 0.9,0.98,1])

0.10       18.0
0.25       23.0
0.50       30.0
0.75      100.0
0.90      200.0
0.98      700.0
1.00    12143.0
Name: students_reached, dtype: float64

In [22]:
sel = pp.na_fill_col(sel, 'students_reached', np.nanmedian)

#### Check that there are no more missing values in feature columns

In [23]:
for x in pp.na_col(sel):
    if x in feature_cols:
        print(x)

#### Discretize numeric features and then get all dummy variables.

In [24]:
# discretize numeric features
bucketdict= {'total_price_including_optional_support': 4, 'students_reached':4}
df_discr = pp.feat_mult_disc(sel, bucketdict, qt=True)

df_discr.total_price_including_optional_support_binned.unique()
df_discr.students_reached_binned.unique()

[(30.0, 100.0], (23.0, 30.0], (0.999, 23.0], (100.0, 12143.0]]
Categories (4, interval[float64]): [(0.999, 23.0] < (23.0, 30.0] < (30.0, 100.0] < (100.0, 12143.0]]

In [25]:
col_to_binary = list(df_discr.columns)
col_to_binary.remove('label')
col_to_binary.remove('date_posted')

In [26]:
col_to_binary

['school_metro',
 'school_charter',
 'school_magnet',
 'primary_focus_subject',
 'primary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'eligible_double_your_impact_match',
 'total_price_including_optional_support_binned',
 'students_reached_binned']

In [27]:
# turn variables into dummies
df_final = pp.feat_binary(df_discr, col_to_binary)
df_final.head()

,date_posted,label,school_metro_rural,school_metro_suburban,school_metro_urban,school_charter_f,school_charter_t,school_magnet_f,school_magnet_t,primary_focus_subject_Applied Sciences,...,eligible_double_your_impact_match_f,eligible_double_your_impact_match_t,"total_price_including_optional_support_binned_(91.999, 345.81]","total_price_including_optional_support_binned_(345.81, 510.5]","total_price_including_optional_support_binned_(510.5, 752.96]","total_price_including_optional_support_binned_(752.96, 164382.84]","students_reached_binned_(0.999, 23.0]","students_reached_binned_(23.0, 30.0]","students_reached_binned_(30.0, 100.0]","students_reached_binned_(100.0, 12143.0]"
0,2013-04-14,0,0,0,1,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,0
1,2012-04-07,0,0,0,1,1,0,1,0,0,...,0,1,1,0,0,0,0,1,0,0
2,2012-01-30,1,0,0,1,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,0
3,2012-10-11,0,0,0,1,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
4,2013-01-08,1,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1


## Run variations of models: 
### Decision trees, KNN, Logistic Regression, Linear SVM, Random forests, Bagging, Boosting

In [28]:
windows = [dt.datetime(2012,1,1), dt.datetime(2012,7,1), dt.datetime(2013,1,1), dt.datetime(2013,7,1), dt.datetime(2014,1,1)]
pred_time = 60 #days
label_col = 'label'
split_col = 'date_posted'
feature_cols= list(df_final.columns)
feature_cols.remove('label')
feature_cols.remove('date_posted')
seed=12345

In [ ]:
models = [
    {'type': 'Dtree', 'clf': pp.dtree_score, 'criteria': ['entropy', 'gini'], 'depth': [10,20,30],'min_leaf': [100, 300,500], 'seed': seed},
    {'type': 'LR', 'clf': pp.lr_score, 'p': ['l1','l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'solver': ['liblinear'], 'seed': seed},
    {'type': 'SVM', 'clf': pp.linsvc_score, 'p': ['l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'seed': seed},
    {'type': 'Bagging_dtree', 'clf': pp.bagging_score, 'n': [10, 50, 100], 'base':[None], 'seed':seed},
    {'type': 'ADABoost_dtree', 'clf': pp.adaboost_score, 'n': [10, 50, 100], 'base':[None], 'seed':seed},
    {'type': 'Random Forest', 'clf': pp.rforest_score, 'n': [10, 50, 100], 'criterion': ['entropy', 'gini'], 'seed': seed},
    {'type': 'KNN', 'clf': pp.knn_score, 'n': [5], 'weights': ['uniform','distance'], 'distance_metric':['minkowski'],'p': [1,2]}
]

#models = [{'type': 'Random Forest', 'clf': rforest_score, 'n': [10, 50, 100], 'criterion': ['entropy', 'gini'], 'seed': seed}]
thresholds = [1, 2, 5, 10, 20,30, 50]


In [ ]:
#this code was run separately and saved in csv for future use, but this is how it is run
resdf = pp.run_models(models, thresholds, windows, df_final, feature_cols, label_col, split_col, pred_time, pred_unit = 'day', filename = './data/finalrun.csv')

In [4]:
resdf = pp.load_csv('./data/finalrun2.csv')
resdf.head()

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
0,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.256917,1,0.442424,0.017241,0.504864,1,2012-01-01,2012-07-01
1,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.256917,2,0.356601,0.027752,0.505220,1,2012-01-01,2012-07-01
2,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.256917,5,0.300364,0.058455,0.505689,1,2012-01-01,2012-07-01
3,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.256917,10,0.279126,0.108644,0.505817,1,2012-01-01,2012-07-01
4,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.256917,20,0.265928,0.207015,0.504720,1,2012-01-01,2012-07-01


## Comparing model results

#### Start by finding models with highest precision, recall, or auc.

##### Models have the highest percision at a threshold of 1%

In [63]:
resdf.sort_values('precision', ascending=False)

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
483,LR,"penalty: l2, c: 1.0, solver: liblinear, seed: ...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
525,SVM,"penalty: l2, c: 100.0, seed: 12345",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
455,LR,"penalty: l1, c: 1.0, solver: liblinear, seed: ...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
462,LR,"penalty: l1, c: 10.0, solver: liblinear, seed:...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
469,LR,"penalty: l1, c: 100.0, solver: liblinear, seed...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
476,LR,"penalty: l2, c: 0.1, solver: liblinear, seed: ...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
490,LR,"penalty: l2, c: 10.0, solver: liblinear, seed:...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
497,LR,"penalty: l2, c: 100.0, solver: liblinear, seed...",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
504,SVM,"penalty: l2, c: 0.1, seed: 12345",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01
511,SVM,"penalty: l2, c: 1.0, seed: 12345",0.315889,1,0.589862,0.018667,0.506337,2,2012-01-01,2013-01-01


##### Models have the highest recall at a threshold of 50%

In [64]:
resdf.sort_values('recall', ascending=False)

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
405,Dtree,"criteria: gini, depth: 10, min_leaf: 500, seed...",0.315889,50,0.323844,0.512615,0.509203,2,2012-01-01,2013-01-01
454,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.315889,50,0.323844,0.512615,0.509203,2,2012-01-01,2013-01-01
342,Dtree,"criteria: entropy, depth: 10, min_leaf: 500, s...",0.315889,50,0.323844,0.512615,0.509203,2,2012-01-01,2013-01-01
468,LR,"penalty: l1, c: 10.0, solver: liblinear, seed:...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01
503,LR,"penalty: l2, c: 100.0, solver: liblinear, seed...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01
461,LR,"penalty: l1, c: 1.0, solver: liblinear, seed: ...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01
475,LR,"penalty: l1, c: 100.0, solver: liblinear, seed...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01
482,LR,"penalty: l2, c: 0.1, solver: liblinear, seed: ...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01
496,LR,"penalty: l2, c: 10.0, solver: liblinear, seed:...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01
489,LR,"penalty: l2, c: 1.0, solver: liblinear, seed: ...",0.315889,50,0.323659,0.512323,0.508990,2,2012-01-01,2013-01-01


##### Decision trees have the highest AUC

In [71]:
aucdf = resdf.sort_values('auc', ascending=False)
aucdf

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
397,Dtree,"criteria: gini, depth: 10, min_leaf: 300, seed...",0.315889,30,0.332617,0.315882,0.511611,2,2012-01-01,2013-01-01
334,Dtree,"criteria: entropy, depth: 10, min_leaf: 300, s...",0.315889,30,0.332617,0.315882,0.511611,2,2012-01-01,2013-01-01
439,Dtree,"criteria: gini, depth: 30, min_leaf: 300, seed...",0.315889,30,0.332156,0.315444,0.511291,2,2012-01-01,2013-01-01
481,LR,"penalty: l2, c: 0.1, solver: liblinear, seed: ...",0.315889,30,0.332156,0.315444,0.511291,2,2012-01-01,2013-01-01
355,Dtree,"criteria: entropy, depth: 20, min_leaf: 300, s...",0.315889,30,0.332156,0.315444,0.511291,2,2012-01-01,2013-01-01
376,Dtree,"criteria: entropy, depth: 30, min_leaf: 300, s...",0.315889,30,0.332156,0.315444,0.511291,2,2012-01-01,2013-01-01
418,Dtree,"criteria: gini, depth: 20, min_leaf: 300, seed...",0.315889,30,0.332156,0.315444,0.511291,2,2012-01-01,2013-01-01
565,ADABoost_dtree,"n: 50, base: None",0.315889,30,0.331849,0.315152,0.511078,2,2012-01-01,2013-01-01
530,SVM,"penalty: l2, c: 100.0, seed: 12345",0.315889,30,0.331849,0.315152,0.511078,2,2012-01-01,2013-01-01
453,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.315889,30,0.331695,0.315007,0.510971,2,2012-01-01,2013-01-01


In [72]:
aucdf.groupby(['train_set_num']).first()

,type,details,baseline,threshold_pct,precision,recall,auc,train_start,test_start
train_set_num,,,,,,,,,
1,SVM,"penalty: l2, c: 0.1, seed: 12345",0.256917,50,0.262864,0.511573,0.507787,2012-01-01,2012-07-01
2,Dtree,"criteria: gini, depth: 10, min_leaf: 300, seed...",0.315889,30,0.332617,0.315882,0.511611,2012-01-01,2013-01-01
3,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.284647,5,0.354167,0.062202,0.508534,2012-01-01,2013-07-01


#### From above, we can see that finding models that do well on a measure is too simplistic. It's better to identify models that do better on precision, recall by a threshold and time period

In [61]:
best_prec = resdf.sort_values('precision', ascending=False)
best_prec.groupby(['train_set_num', 'threshold_pct']).nth([0])

type  \
train_set_num threshold_pct          
1             1                 LR   
              2                SVM   
              5                 LR   
              10                LR   
              20                LR   
              30                LR   
              50                LR   
2             1                 LR   
              2                SVM   
              5              Dtree   
              10               SVM   
              20             Dtree   
              30             Dtree   
              50             Dtree   
3             1              Dtree   
              2              Dtree   
              5                 LR   
              10                LR   
              20                LR   
              30                LR   
              50                LR   

                                                                       details  \
train_set_num threshold_pct                                                      
1             1              penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              2                               penalty: l2, c: 0.1, seed: 12345   
              5              penalty: l1, c: 1.0, solver: liblinear, seed: ...   
              10             penalty: l2, c: 1.0, solver: liblinear, seed: ...   
              20             penalty: l1, c: 1.0, solver: liblinear, seed: ...   
              30             penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              50             penalty: l2, c: 0.1, solver: liblinear, seed: ...   
2             1              penalty: l2, c: 1.0, solver: liblinear, seed: ...   
              2                             penalty: l2, c: 100.0, seed: 12345   
              5              criteria: gini, depth: 30, min_leaf: 300, seed...   
              10                             penalty: l2, c: 10.0, seed: 12345   
              20             criteria: gini, depth: 10, min_leaf: 300, seed...   
              30             criteria: gini, depth: 10, min_leaf: 300, seed...   
              50             criteria: entropy, depth: 10, min_leaf: 500, s...   
3             1              criteria: entropy, depth: 30, min_leaf: 300, s...   
              2              criteria: entropy, depth: 30, min_leaf: 300, s...   
              5              penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              10             penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              20             penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              30             penalty: l1, c: 10.0, solver: liblinear, seed:...   
              50             penalty: l1, c: 0.1, solver: liblinear, seed: ...   

                             baseline  precision    recall       auc  \
train_set_num threshold_pct                                            
1             1              0.256917   0.481818  0.018777  0.505897   
              2              0.256917   0.379363  0.029523  0.506412   
              5              0.256917   0.311893  0.060699  0.507199   
              10             0.256917   0.283981  0.110534  0.507088   
              20             0.256917   0.267597  0.208314  0.505594   
              30             0.256917   0.264563  0.308928  0.506007   
              50             0.256917   0.262864  0.511573  0.507787   
2             1              0.315889   0.589862  0.018667  0.506337   
              2              0.315889   0.474654  0.030042  0.507344   
              5              0.315889   0.372350  0.058918  0.506530   
              10             0.315889   0.353293  0.111856  0.508656   
              20             0.315889   0.338401  0.214234  0.510416   
              30             0.315889   0.332617  0.315882  0.511611   
              50             0.315889   0.323844  0.512615  0.509203   
3             1              0.284647   0.576923  0.020283  0.507182   
              2              0.284647   0.43

In [60]:
resdf.groupby(['train_set_num', 'threshold_pct']).agg({'precision' : 'max', 'recall' : 'max', 'auc': 'max'})

precision    recall       auc
train_set_num threshold_pct                               
1             1               0.481818  0.018777  0.505897
              2               0.379363  0.029523  0.506412
              5               0.311893  0.060699  0.507199
              10              0.283981  0.110534  0.507088
              20              0.267597  0.208314  0.505594
              30              0.264563  0.308928  0.506007
              50              0.262864  0.511573  0.507787
2             1               0.589862  0.018667  0.506337
              2               0.474654  0.030042  0.507344
              5               0.372350  0.058918  0.506530
              10              0.353293  0.111856  0.508656
              20              0.338401  0.214234  0.510416
              30              0.332617  0.315882  0.511611
              50              0.323844  0.512615  0.509203
3             1               0.576923  0.020283  0.507182
              2               0.433749  0.030465  0.507320
              5               0.354167  0.062202  0.508534
              10              0.311071  0.109290  0.506489
              20              0.295030  0.207286  0.505099
              30              0.292528  0.308304  0.505806
              50              0.285501  0.501511  0.501048

#### Choose model for the 5% threshold (target percent of population)

In [11]:
t_5 = resdf[resdf['threshold_pct']==5].sort_values('precision', ascending=False)
t_5.groupby('train_set_num').nth([0,1])

,type,details,baseline,threshold_pct,precision,recall,auc,train_start,test_start
train_set_num,,,,,,,,,
1,LR,"penalty: l1, c: 1.0, solver: liblinear, seed: ...",0.256917,5,0.311893,0.060699,0.507199,2012-01-01,2012-07-01
1,LR,"penalty: l2, c: 0.1, solver: liblinear, seed: ...",0.256917,5,0.311893,0.060699,0.507199,2012-01-01,2012-07-01
2,Dtree,"criteria: entropy, depth: 20, min_leaf: 300, s...",0.315889,5,0.372350,0.058918,0.506530,2012-01-01,2013-01-01
2,Dtree,"criteria: entropy, depth: 30, min_leaf: 300, s...",0.315889,5,0.372350,0.058918,0.506530,2012-01-01,2013-01-01
3,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.284647,5,0.354167,0.062202,0.508534,2012-01-01,2013-07-01
3,LR,"penalty: l2, c: 0.1, solver: liblinear, seed: ...",0.284647,5,0.354167,0.062202,0.508534,2012-01-01,2013-07-01
